In [16]:
import littletable as lt
from IPython.display import HTML, display

# shortcut for displaying tables as HTML inside Notebook
def _HTML(tbl, fields='*'):
    display(HTML(tbl.as_html(fields)))

In [17]:
# import from csv, convert string fields to ints and floats
# (convert elevation from meters to feet)
# us_ppl.csv is an extract of population data from the GeoNames database of US place 
# names (www.geonames.org), licensed under the Creative Commons Attribution 4.0 License
def str_meters_to_int_feet(s: str) -> int:
    return int(int(s)*328/100+0.5)

us_ppl = lt.Table().csv_import(
    'us_ppl.csv', 
    transforms={'pop': int, 'lat': float, 'long': float, 'elev': str_meters_to_int_feet}
)

{'len': 25860, 'name': 'us_ppl.csv', 'fields': ['tz', 'name', 'feature', 'long', 'countycode', 'state', 'pop', 'elev', 'lat', 'featclass', 'wikilink', 'id'], 'indexes': [], 'created': datetime.datetime(2024, 3, 22, 0, 36, 37, 168369, tzinfo=datetime.timezone.utc), 'modified': datetime.datetime(2024, 3, 22, 0, 36, 37, 284952, tzinfo=datetime.timezone.utc), 'last_import': datetime.datetime(2024, 3, 22, 0, 36, 37, 284952, tzinfo=datetime.timezone.utc)}


In [18]:
# define index by state, since we will do many analyses and pivots by state
# (may only pivot on indexed attributes)
us_ppl.create_index('state')

In [19]:
# display summary info about this table
us_ppl.info()

{'len': 25860,
 'name': 'us_ppl.csv',
 'fields': ['tz',
  'name',
  'feature',
  'long',
  'countycode',
  'state',
  'pop',
  'elev',
  'lat',
  'featclass',
  'wikilink',
  'id'],
 'indexes': [('state', False)],
 'created': datetime.datetime(2024, 3, 22, 0, 36, 37, 168369, tzinfo=datetime.timezone.utc),
 'modified': datetime.datetime(2024, 3, 22, 0, 36, 37, 284952, tzinfo=datetime.timezone.utc),
 'last_import': datetime.datetime(2024, 3, 22, 0, 36, 37, 284952, tzinfo=datetime.timezone.utc)}

In [20]:
# drop columns that are not so interesting
us_ppl = us_ppl.select("-countycode -featclass -feature -tz -wikilink")

In [21]:
# display a table of the top 20 places in Texas by population
_HTML(
    us_ppl.by.state['TX'].orderby("pop desc")[:20],
    "id name lat long elev pop"
)

id,name,lat,long,elev,pop
4699066,Houston,29.76328,-95.36327,39,2027712
4726206,San Antonio,29.42412,-98.49363,649,1256810
4684888,Dallas,32.78306,-96.80667,420,1211704
4671654,Austin,30.26715,-97.74306,489,678368
4691930,Fort Worth,32.72541,-97.32085,653,618119
5520993,El Paso,31.75872,-106.48693,3716,563662
4671240,Arlington,32.73569,-97.10807,604,332969
4683416,Corpus Christi,27.80058,-97.39638,7,277454
4719457,Plano,33.01984,-96.69889,666,253372
4693003,Garland,32.91262,-96.63888,551,217219


In [22]:
# what are all the states in the input data file
us_ppl.by.state.keys()

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [23]:
# create a pivot table by state
piv = us_ppl.pivot("state")

In [24]:
# compute the total population by state, and output as a table
_HTML(piv.summarize(lambda recs: sum(r.pop for r in recs), "population"))

state,population
AK,563056
AL,2897156
AR,1812861
AZ,5222978
CA,33430472
CO,3849638
CT,3197085
DC,552433
DE,351776
FL,12887696


In [25]:
# compute the total population by state, and output as a table
_HTML(piv.as_table(sum, 'pop').orderby("pop desc"), "state pop")

state,pop
CA,33430472
TX,18076558
NY,15576116
FL,12887696
IL,11267435
OH,8024440
MA,7596206
PA,6851266
NJ,6465721
MI,6181472


In [26]:
# add a computed field 'elev000', elevation rounded down by 1000's
us_ppl.add_field('elev000', lambda rec: int(rec.elev/1000)*1000, 0)

In [27]:
# create index on elev000 and pivot
us_ppl.create_index('elev000')
piv = us_ppl.pivot('elev000')

In [28]:
# create a table of the summary counts of the pivot, add a label field and display as a table
ppl_by_elev = piv.as_table(sum, 'pop')
ppl_by_elev.add_field("elevation", lambda  rec: f"{rec.elev000}-{rec.elev000+1000}")
_HTML(
    ppl_by_elev, 
    "elevation pop"
)

elevation,pop
0-1000,184718782
1000-2000,22464911
2000-3000,4703076
3000-4000,2720285
4000-5000,4678777
5000-6000,3281299
6000-7000,1103386
7000-8000,210123
8000-9000,36772
9000-10000,13202


In [29]:
# summarize population by state and elevation
piv = us_ppl.pivot('state elev000')

In [30]:
# display table of residents in each state by elevation
_HTML(
    piv.summarize(lambda recs: sum(r.pop for r in recs))
)

0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000
AK,548686,11696,2512,162,0,0,0,0,0,0
AL,2832975,64181,0,0,0,0,0,0,0,0
AR,1478252,334280,329,0,0,0,0,0,0,0
AZ,435195,3368090,908140,128847,136507,134613,104211,7375,0,0
CA,29785419,2493046,801137,178226,63354,49358,52243,7689,0,0
CO,0,0,0,33791,609903,2483419,549769,121815,33341,13038
CT,3185846,11239,0,0,0,0,0,0,0,0
DC,552433,0,0,0,0,0,0,0,0,0
DE,351776,0,0,0,0,0,0,0,0,0
FL,12887696,0,0,0,0,0,0,0,0,0


In [31]:
# what is the average resident elevation by state?
piv = us_ppl.pivot('state')
_HTML(
    piv.summarize(
        lambda recs: int(sum(r.pop*r.elev for r in recs)/sum(r.pop for r in recs)),
        col_label='ave elevation'
    )
)

state,ave elevation
AK,175
AL,476
AR,513
AZ,1718
CA,398
CO,5496
CT,173
DC,23
DE,86
FL,34
